<a href="https://colab.research.google.com/github/GizawAAiT/Deep-Learning/blob/main/DL_Lab_4_Gizaw_Dagne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [261]:
# Import pytorch
import torch

### Creating  Delse Layers (Class)

In [262]:
class DenseLayer:
  def __init__(self, features = 0, neurons= 0):
    self.features= features
    self.neurons = neurons
    self.weights = torch.rand(features, neurons)
    self.bias = torch.rand(neurons)

  # Do forward pass
  def forward(self, inputs):
    self.output = inputs@self.weights + self.bias

  # relu activation
  def setReluActivation(self, x):
    self.relu = torch.max(x, torch.tensor(0))

  # sigmoid activation
  def setSigmoidActivation(self, x):
    self.sigmoid = 1/(1 + torch.exp(-x))

  # softmax activation
  def setSoftmaxActivation(self, x):
    exp_values = torch.exp(x - torch.max(x, axis=1, keepdim=True).values)
    # Normalize them for each sample
    self.softmax = exp_values / torch.sum(exp_values, axis=1, keepdim=True)

  def setLinearActivation(self, x):
    self.linear = x

  # categorical loss cross entropy C.L.C.E
  def categoricalCrossentropy(self, y_true, y_pred):
    samples = len(y_pred)
    # Clip data to prevent division by 0
    # Clip both sides to not drag mean towards any value
    y_pred_clipped = torch.clip(y_pred, 1e-8, 1 - 1e-8)
    # only if categorical labels
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true]
    # Mask values - only for one-hot encoded labels
    elif len(y_true.shape) == 2:
      correct_confidences = torch.sum(y_pred_clipped * y_true, axis=1)
    log_loss = -torch.log(correct_confidences)
    data_loss = torch.mean(log_loss)
    return data_loss

  def accuracy(self, y_pred, y_true):
    predictions = torch.argmax(y_pred, axis=1)
    if len(y_true.shape) == 2:
      y_true = torch.argmax(y_true, axis=1)
    acc = torch.mean((predictions == y_true).float())
    return acc


In [263]:
sample_layer = DenseLayer()

# sample data to calculate loss and accuracy:
softmax_outputs = torch.tensor([[0.7, 0.1, 0.2], [0.1, 0.5, 0.4],[0.02, 0.9, 0.08]])
class_targets = torch.tensor([[1, 0, 0], [0, 1, 0], [1, 0, 0]])

loss = sample_layer.categoricalCrossentropy(class_targets, softmax_outputs)
accuracy = sample_layer.accuracy(softmax_outputs, class_targets)
print(f'Loss = {loss}\nAccuracy = {accuracy}')

Loss = 1.653948426246643
Accuracy = 0.6666666865348816


## Example 1
### Preparing Dataset


In [264]:
import numpy as np
from sklearn.datasets import load_iris

In [265]:
# Load the Iris dataset from scikit-learn
iris = load_iris()
X = iris.data
y = iris.target


In [266]:
# Convert the NumPy arrays to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

In [267]:
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Feature names:", iris.feature_names)
print("Class names:", iris.target_names)
print(X[:5])
print(y[:5])

X shape: torch.Size([150, 4])
y shape: torch.Size([150])
Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Class names: ['setosa' 'versicolor' 'virginica']
tensor([[5.0999999046, 3.5000000000, 1.3999999762, 0.2000000030],
        [4.9000000954, 3.0000000000, 1.3999999762, 0.2000000030],
        [4.6999998093, 3.2000000477, 1.2999999523, 0.2000000030],
        [4.5999999046, 3.0999999046, 1.5000000000, 0.2000000030],
        [5.0000000000, 3.5999999046, 1.3999999762, 0.2000000030]])
tensor([0, 0, 0, 0, 0])


In [268]:
class ClassificationModel():

  def __init__(self, num_of_features, num_of_class):
    # creating the model
    self.dense1 = DenseLayer(num_of_features,16)
    self.dense2 = DenseLayer(16, 16)
    self.output_layer = DenseLayer(16, num_of_class)

  def model(self, X, y):
    self.y = y
    # forward pass
    self.dense1.forward(X)
    self.dense1.setReluActivation(self.dense1.output)
    self.dense2.forward(self.dense1.relu)
    self.dense2.setReluActivation(self.dense2.output)
    self.output_layer.forward(self.dense2.relu)
    self.output_layer.setSoftmaxActivation(self.output_layer.output)

  def loss_and_accuracy(self):
    self.loss = self.output_layer.categoricalCrossentropy(self.y, self.output_layer.softmax)
    self.accuracy = self.output_layer.accuracy(self.output_layer.softmax, self.y)

In [269]:
test = ClassificationModel(4, 3)

In [270]:
test.model(X, y)
test.loss_and_accuracy()

print(f'''loss = {test.loss}\nAccuracy = {test.accuracy}''')

loss = 12.280454635620117
Accuracy = 0.3333333432674408


##  How can we adjust the weights and biases to decrease the loss?



### Option 1). Randomly changing the weights, checking the loss, and repeating this until the lowest loss found.

In [271]:
test_iris = ClassificationModel(4, 3)
torch.set_printoptions(precision=10)
lowest_loss = torch.tensor(99999999)
ln_rate = 0.02

In [272]:
for iteration in range(1000):

  # Perform a forward pass
  test_iris.model(X,y)
  test_iris.loss_and_accuracy()
  loss = test_iris.loss
  accuracy = test_iris.accuracy

  # If loss is smaller - print and save weights and biases aside
  if loss < lowest_loss:
    print('New set of weights found, iteration:', iteration, 'loss:', loss, 'acc:', accuracy)
    best_dense1_weights = test_iris.dense1.weights
    best_dense1_biases = test_iris.dense1.bias
    best_dense2_weights = test_iris.dense2.weights
    best_dense2_biases = test_iris.dense2.bias
    best_output_layer_weights = test_iris.output_layer.weights
    best_output_layer_biases = test_iris.output_layer.bias
    lowest_loss = loss

  # Generate a new set of weights for iteration
  test_iris.dense1.weights = ln_rate * torch.rand(4, 16)
  test_iris.dense1.biases = ln_rate * torch.rand(1, 16)
  test_iris.dense2.weights = ln_rate * torch.rand(16, 16)
  test_iris.dense2.biases = ln_rate * torch.rand(1, 16)

  test_iris.output_layer.weights = ln_rate * torch.rand(16, 3)
  test_iris.output_layer.biases = ln_rate * torch.rand(1, 3)

New set of weights found, iteration: 0 loss: tensor(12.2804546356) acc: tensor(0.3333333433)
New set of weights found, iteration: 1 loss: tensor(1.1324729919) acc: tensor(0.3333333433)
New set of weights found, iteration: 2 loss: tensor(1.1310031414) acc: tensor(0.3333333433)
New set of weights found, iteration: 4 loss: tensor(1.1292535067) acc: tensor(0.3333333433)
New set of weights found, iteration: 6 loss: tensor(1.1283520460) acc: tensor(0.3333333433)
New set of weights found, iteration: 9 loss: tensor(1.1278623343) acc: tensor(0.3333333433)
New set of weights found, iteration: 10 loss: tensor(1.1272079945) acc: tensor(0.3333333433)
New set of weights found, iteration: 14 loss: tensor(1.1271808147) acc: tensor(0.3333333433)
New set of weights found, iteration: 25 loss: tensor(1.1250435114) acc: tensor(0.3333333433)
New set of weights found, iteration: 58 loss: tensor(1.1245460510) acc: tensor(0.3333333433)
New set of weights found, iteration: 74 loss: tensor(1.1245013475) acc: ten

In [273]:
print('loss:', test_iris.loss)
print("Accuracy:", test_iris.accuracy)

loss: tensor(1.1306099892)
Accuracy: tensor(0.3333333433)


#### Option 2:) Instead of setting parameters with randomly-chosen values each iteration, apply a fraction of these values to parameters.

In [274]:
test_iris = ClassificationModel(4, 3)
lowest_loss = torch.tensor(99999999)
lr_rate = 0.05

In [275]:
for iteration in range(10000):

  # Perform a forward pass
  test_iris.model(X,y)
  test_iris.loss_and_accuracy()
  loss = test_iris.loss
  accuracy = test_iris.accuracy

  # If loss is smaller - print and save weights and biases aside
  if loss < lowest_loss:
    print('New set of weights found, iteration:', iteration, 'loss:', loss, 'acc:', accuracy)
    best_dense1_weights = test_iris.dense1.weights
    best_dense1_biases = test_iris.dense1.bias
    best_dense2_weights = test_iris.dense2.weights
    best_dense2_biases = test_iris.dense2.bias
    best_output_layer_weights = test_iris.output_layer.weights
    best_output_layer_biases = test_iris.output_layer.bias
    lowest_loss = loss

  else:
    test_iris.dense1.weights = best_dense1_weights
    test_iris.dense1.biases = best_dense1_biases
    test_iris.dense2.weights = best_dense2_weights
    test_iris.dense2.biases = best_dense2_biases
    test_iris.output_layer.weights = best_output_layer_weights
    test_iris.output_layer.biases = best_output_layer_biases

  # Generate a new set of weights for iteration
  test_iris.dense1.weights += lr_rate * torch.rand(4, 16)
  test_iris.dense1.bias += lr_rate * torch.rand(16)
  test_iris.dense2.weights += lr_rate * torch.rand(16, 16)
  test_iris.dense2.bias += lr_rate * torch.rand(16)
  test_iris.output_layer.weights += lr_rate * torch.rand(16, 3)
  test_iris.output_layer.bias += lr_rate * torch.rand(3)

New set of weights found, iteration: 0 loss: tensor(12.2804546356) acc: tensor(0.3333333433)
New set of weights found, iteration: 12 loss: tensor(6.7108340263) acc: tensor(0.0533333346)


#### Option 3:  using optimization
(Assignment)

**Forward and Backward propagation**

*   Use 2 features in the input layer, 1 hidden layer with 4 neurons, and an output layer with 2 neurons.
*   Use sigmoid activation in the hidden layer and linear activation in the output layer.
*   Assume the task is regression task and use MSE for the loss function.





In [331]:
# create sample dataset
X = torch.rand(32, 2)-.5 #distribute the random values between (-0.5 and +0.5).


In [345]:
# create the hiddnel layer and the output layer.
hidden_layer = DenseLayer(2, 4)
output_layer = DenseLayer(4, 2)

In [338]:
# create sample true value
y_true = torch.rand(32, 2) # The output layer has 2 neurons.

In [310]:
# define the forward pass
def forward_pass(X):
  hidden_layer.forward(X)
  hidden_layer.setSigmoidActivation(hidden_layer.output)
  output_layer.forward(hidden_layer.output)
  output_layer.setLinearActivation(output_layer.output)
  # print(f'linear : {output_layer.linear}')

  return output_layer.linear


In [333]:

def back_prop(fp):
    lr = torch.tensor(0.01)
    # Assuming y is the target for the regression task
    error = fp - y_true

    # Backpropagation for the output layer
    output_layer.weights -= lr * (hidden_layer.sigmoid.T @ error)
    output_layer.bias -= lr * torch.sum(error, dim=0)

    # Backpropagation for the hidden layer
    hidden_error = (error @ output_layer.weights.T) * hidden_layer.sigmoid * (1 - hidden_layer.sigmoid)
    hidden_layer.weights -= lr * (X.T @ hidden_error)
    hidden_layer.bias -= lr * torch.sum(hidden_error, dim=0)

In [312]:
def error_calculation(y_true, y_pred):
  return torch.mean(0.5*(y_true - y_pred)**2)

In [343]:
loss = 0.099

In [346]:
y_pred = forward_pass(X)
err = error_calculation(y_true, y_pred)
print("Initial loss:", err)
print("Initial prediction:", y_pred)
while err > loss:
  back_prop(y_pred)
  y_pred = forward_pass(X)
  err = error_calculation(y_true, y_pred)
print("\n\n\nFinal loss:", err)
print("Final prediction:",y_pred)
print("Target value:",y_true)

Initial loss: tensor(0.7880818844)
Initial prediction: tensor([[1.4152278900, 1.2860889435],
        [1.7212860584, 1.4522829056],
        [2.1913914680, 1.7633730173],
        [1.6524333954, 1.4323304892],
        [2.4005827904, 1.9116258621],
        [1.2251117229, 1.1629550457],
        [1.6194715500, 1.3856616020],
        [1.7657186985, 1.4922084808],
        [0.8690660596, 0.9686335921],
        [2.0537621975, 1.6681077480],
        [1.2621660233, 1.1773351431],
        [2.0370202065, 1.6802566051],
        [1.2974567413, 1.2299078703],
        [2.1763648987, 1.7444640398],
        [1.5875909328, 1.3703732491],
        [0.9450339675, 1.0167793036],
        [2.3470919132, 1.8322324753],
        [1.5318077803, 1.3634107113],
        [2.3729684353, 1.8826054335],
        [1.4526010752, 1.3208800554],
        [2.3735969067, 1.8725064993],
        [1.8776748180, 1.5931897163],
        [2.0083017349, 1.6729229689],
        [2.5812366009, 1.9835014343],
        [1.2393946648, 1.18238091